In [4]:

TASK = "classification"
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (
    accuracy_score, classification_report, r2_score, mean_squared_error
)
import warnings; warnings.filterwarnings("ignore", category=UserWarning)

def is_classification_task(TASK): return TASK.lower().startswith("class")

X = np.random.rand(100, 10)
if is_classification_task(TASK):
    y = np.random.randint(0, 2, 100)
else:
    y = np.random.rand(100)


stratify_arg = None
if is_classification_task(TASK):

    try:

        stratify_arg = y
    except Exception:
        stratify_arg = None

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=stratify_arg
)


if is_classification_task(TASK):
    base_model = RandomForestClassifier(
        n_estimators=100, random_state=42, n_jobs=-1
    )
    scoring = "accuracy"
else:
    base_model = RandomForestRegressor(
        n_estimators=100, random_state=42, n_jobs=-1
    )
    scoring = "r2"


baseline_cv = cross_val_score(base_model, X_train, y_train, cv=3, scoring=scoring, n_jobs=-1)
baseline_cv_mean = baseline_cv.mean()

# Fit baseline and evaluate on test
base_model.fit(X_train, y_train)
if is_classification_task(TASK):
    y_pred_test_base = base_model.predict(X_test)
    baseline_test_metric = accuracy_score(y_test, y_pred_test_base)
else:
    y_pred_test_base = base_model.predict(X_test)
    baseline_test_metric = r2_score(y_test, y_pred_test_base)

print("=== Baseline RandomForest ===")
print(f"CV ({scoring}) mean: {baseline_cv_mean:.4f}")
print(f"Test ({scoring}): {baseline_test_metric:.4f}\n")


from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}

Estimator = RandomForestClassifier if is_classification_task(TASK) else RandomForestRegressor
grid = GridSearchCV(
    Estimator(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    scoring=scoring,
    cv=3,
    n_jobs=-1,
    verbose=0,
    refit=True
)

grid.fit(X_train, y_train)

print("=== Grid Search Results ===")
print(f"Best params: {grid.best_params_}")
print(f"Best CV ({scoring}) mean: {grid.best_score_:.4f}\n")


best_model = grid.best_estimator_
y_pred_test_best = best_model.predict(X_test)

if is_classification_task(TASK):
    tuned_test_metric = accuracy_score(y_test, y_pred_test_best)
    print("=== Test Set Performance (Tuned) ===")
    print(f"Accuracy: {tuned_test_metric:.4f}")
    print("\nClassification report:\n")
    print(classification_report(y_test, y_pred_test_best))
else:
    tuned_test_metric = r2_score(y_test, y_pred_test_best)
    mse = mean_squared_error(y_test, y_pred_test_best)
    print("=== Test Set Performance (Tuned) ===")
    print(f"R^2: {tuned_test_metric:.4f}")
    print(f"MSE: {mse:.4f}")


print("\n=== Before vs After (summary) ===")
print(f"CV ({scoring})   - Baseline: {baseline_cv_mean:.4f} | Tuned: {grid.best_score_:.4f}")
print(f"Test ({scoring}) - Baseline: {baseline_test_metric:.4f} | Tuned: {tuned_test_metric:.4f}")

=== Baseline RandomForest ===
CV (accuracy) mean: 0.4744
Test (accuracy): 0.5500

=== Grid Search Results ===
Best params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50}
Best CV (accuracy) mean: 0.4986

=== Test Set Performance (Tuned) ===
Accuracy: 0.5500

Classification report:

              precision    recall  f1-score   support

           0       0.50      0.44      0.47         9
           1       0.58      0.64      0.61        11

    accuracy                           0.55        20
   macro avg       0.54      0.54      0.54        20
weighted avg       0.55      0.55      0.55        20


=== Before vs After (summary) ===
CV (accuracy)   - Baseline: 0.4744 | Tuned: 0.4986
Test (accuracy) - Baseline: 0.5500 | Tuned: 0.5500
